# Algoritmos de búsqueda
**Facundo A. Lucianna - Inteligencia Artificial - CEIA - FIUBA**

Un algoritmo de búsqueda toma un problema de búsqueda como entrada y retorna una solución, o una indicación de falla.

La idea es buscar un camino que llegue al estado objetivo. Para ello vamos a construir un arbol que irá avanzando por estados del grafo hasta llegar al estado objetivo.  

![arbol_de_hanoi](./img/tree_hanoi.png)

Cada nodo del árbol corresponde a un **estado** y las aristas corresponde a una **acción**. Importante, el árbol **NO** es el grafo de estados. El grafo describe todo el set de estados, y las acciones que llevan de un lado a otro. El árbol describe el camino entre estos estados, para alcanzar el objetivo. 

Para poder aplicar los algoritmos, debemos definir las estructuras de datos para hacer seguimiento del árbol. 

## Nodos del árbol

Los nodos del árbol son representados con los siguientes componentes:

- **State**: El estado del espacio de estados, que corresponde el nodo.
- **Node Parent**: El nodo en el árbol de búsqueda que ha generado al nodo.
- **Action**: La acción que se aplicará al padre para generar el nodo.
- **Path-Cost**: El costo de un camino desde el nodo inicial al nodo.

Definamos al **Problema** de la torre de Hanoi:

In [1]:
from aima_libs.hanoi_states import ProblemHanoi, StatesHanoi

initial_state = StatesHanoi([5, 4, 3, 2, 1], [], [], max_disks=5)
goal_state = StatesHanoi([], [], [5, 4, 3, 2, 1], max_disks=5)

problem = ProblemHanoi(initial=initial_state, goal=goal_state)

Empezamos con la primera estructura, acá lo implementamos con la clase `NodeHanoi`. Esta clase tiene implementado:

Atributos:
* `state`: Es el estado que el nodo tiene adentro. Representa un estado particular de ubicación de los discos.
* `parent`: Es el nodo padre de este nodo. Si el nodo es la raíz, es `None`.
* `action`: Es la acción que se aplicó al padre para llegar a este nodo. Si es la raíz, es `None`.
* `path_cost`: Es el costo del camino desde la raíz del árbol hasta este nodo.
* `depth`: Es la profundidad del árbol en que se encuentra. Si es la raíz, es cero, si es un hijo de la raíz es igual a 1, etc.

Métodos:
* `child_node`:  Genera el nodo hijo a partir de una acción.
* `expand`: Expande la frontera de este nodo, devolviendo los nodos que son hijos del nodo que se aplica `expand`.
* `solution`: Retorna en una lista la secuencia de acciones que van desde la raíz hasta este nodo.
* `path`: Retorna una lista de nodos que van desde la raíz hasta este nodo.
* `generate_solution_for_simulator`: Este método permite obtener una salida para ver una simulación usando PyGame, en otro notebook vamos a profundizar.

Además tiene implementada métodos que nos permite hacer diferentes operaciones en Python:

* Podemos comparar dos nodos si son iguales (haciendo `node1 == node2`)
* Podemos preguntar si un nodo es mayor a otro (haciendo `node1 > node2`), esto significa si el costo acumulado de un costo es mayor a otro.
* Tenemos una representación en string del estado, y es por eso que cuando hacemos `print()` se observa que estado tiene dentro del nodo con el texto `<Node >`.
* También podemos obtener un hash del estado, esto funciona si hacemos `hash(estado)`


In [2]:
from aima_libs.tree_hanoi import NodeHanoi

In [3]:
# Definimos la raíz del arbol
root = NodeHanoi(state=initial_state)

In [4]:
print("El arbol tiene como raíz a:")
print(root)

El arbol tiene como raíz a:
<Node HanoiState: 5 4 3 2 1 |  | >


Desde un nodo y definido el problema, podemos encontrar la frontera, la cual es la separación del grafo que ya fue explorada por el algoritmo de búsqueda y aquella que no.

![frontera_en_arbol_de_hanoi](./img/tree_hanoi_frontier.png)

Expandamos la frontera del nodo raíz:

In [5]:
lista_nodos_fronteras = root.expand(problem=problem)

In [6]:
for nodos in lista_nodos_fronteras:
    print(nodos)

<Node HanoiState: 5 4 3 2 | 1 | >
<Node HanoiState: 5 4 3 2 |  | 1>


Los estados que tienen estos nuevos nodos son:

![state](./img/state_hanoi5.png)

Entonces, el árbol para este problema nos queda:

![tree](./img/tree_hanoi2.png)

Es decir, se generan dos nodos nuevos con los siguientes estados:

- El disco verde (el disco más chico) se movía a la varilla del medio.
- El disco verde (el disco más chico) se movía a la varilla derecha.

Nos quedemos con el nodo con el estado que tiene el disco verde en el medio:

In [7]:
next_node = lista_nodos_fronteras[0]

Vemos su estado:

In [8]:
next_node.state

HanoiState: 5 4 3 2 | 1 | 

Vemos quien es el padre, que debería ser la raíz:

In [9]:
next_node.parent

<Node HanoiState: 5 4 3 2 1 |  | >

Que costo acumulado desde la raíz hasta este nodo:

In [10]:
next_node.path_cost

1.0

Y en qué profundidad del árbol estamos:

In [11]:
next_node.depth

1

Expandamos ahora la frontera de este nodo:

In [12]:
lista_nodos_fronteras2 = next_node.expand(problem=problem)

In [13]:
for nodos in lista_nodos_fronteras2:
    print(nodos)

<Node HanoiState: 5 4 3 | 1 | 2>
<Node HanoiState: 5 4 3 2 1 |  | >
<Node HanoiState: 5 4 3 2 |  | 1>


Veamos como quedó ahora el árbol:

![tree](./img/tree_hanoi3.png)

Observamos lo siguiente. Se generan tres nuevos nodos en la frontera. De estos, hay dos que llaman la atención. 

- Hay un nodo que tiene el estado que es igual al del padre, esto es porque si movemos el disco verde de vuelta a la varilla de la izquierda, "retornamos" al estado inicial.
- Hay un nodo que tiene el mismo estado del segundo nodo que se obtuvo del padre.

Esto es importante destacar, múltiples nodos pueden tener el mismo estado, pero el costo para llegar a ese nodo y la secuencia desde la raíz hasta ese nodo va a ser diferentes.

Veamos ahora el nodo del estado que no está en repetido:

In [14]:
next_node2 = lista_nodos_fronteras2[0]

Vemos su estado:

In [15]:
next_node2.state

HanoiState: 5 4 3 | 1 | 2

Vemos quien es el padre:

In [16]:
next_node2.parent

<Node HanoiState: 5 4 3 2 | 1 | >

Que costo acumulado desde la raíz hasta este nodo:

In [17]:
next_node2.path_cost

2.0

Y en qué profundidad del árbol estamos:

In [18]:
next_node2.depth

2

Veamos el camino desde la raíz hasta este nodo:

In [19]:
for nodos in next_node2.path():
    print(nodos)

<Node HanoiState: 5 4 3 2 1 |  | >
<Node HanoiState: 5 4 3 2 | 1 | >
<Node HanoiState: 5 4 3 | 1 | 2>


Y qué acciones se aplicaron desde el inicio hasta este nodo:

In [20]:
for nodos in next_node2.solution():
    print(nodos)

Move disk 1 from 1 to 2
Move disk 2 from 1 to 3


## Colas

La pregunta es, cómo hacemos para elegir cuál nodo seleccionar para explorar la frontera? Para ellos necesitamos de una estructura de datos que nos permita explorar la frontera. Esta estructura es vital para el algoritmo de búsqueda dado que nos permite seleccionar qué modo vamos a expandir primero. Como vimos en los videos, elegir el tipo de estructura para expandir es lo que define el tipo de algoritmo. 

La frontera se expande usando **colas**, la cual tenemos 3 tipos:

- Una cola **FIFO** (primero entra, primero sale) que toma los nodos en el mismo modo que se agregan.
- Una cola **LIFO** (último en salir, sale primero… o stack) quita el nodo más reciente.
- Una **cola prioritaria** que primero quita nodos con el mínimo costo de acuerdo con una función de evaluación f. 

### FIFO

Veamos una implementación de cola FIFO usando una lista:

In [21]:
fifo = []

La implementación debe incorporar la siguiente funciones

- **Add(Frontier)**: Inserta el nodo en su correspondiente lugar de la cola. En el caso de la FIFO, inserta a los nodos en la medida que van llegando usando el método `insert()`.

In [22]:
fifo.insert(0, lista_nodos_fronteras[0])
fifo.insert(0, lista_nodos_fronteras[1])

for nodos in fifo:
    print(nodos)

<Node HanoiState: 5 4 3 2 |  | 1>
<Node HanoiState: 5 4 3 2 | 1 | >


- **Is-empty(frontier)**: Retorna `True` si no hay nodos en la frontera. En el caso de la implementación de una lista, podemos preguntar si la cantidad de elemento es cero.

In [23]:
print("La cola está vacia?")
if len(fifo) == 0:
    print("La cola esta vacía")
else:
    print("La cola tiene elementos")

La cola está vacia?
La cola tiene elementos


- **Pop(frontier)**: Quita el primer nodo en la cola. Con las listas tenemos el método `pop()`.

In [24]:
new_node = fifo.pop()
print(f"El nodo que sacamos es {new_node}")

El nodo que sacamos es <Node HanoiState: 5 4 3 2 | 1 | >


Podemos ver que una vez sacado el nodo, el mismo no está en la fila, y que particularmente sacamos el primer nodo que entró:

In [25]:
print("Los nodos que quedan en la fila son:")
print(fifo)

Los nodos que quedan en la fila son:
[<Node HanoiState: 5 4 3 2 |  | 1>]


Si expandimos la frontera del nuevo nodo que tenemos, podemos agregarlo a la cola:

In [26]:
lista_nodos_fronteras2 = new_node.expand(problem=problem)

# Insertamos a los nodos que son frontera de la raíz en el orden que nos fue presentado:
fifo.insert(0, lista_nodos_fronteras2[0])
fifo.insert(0, lista_nodos_fronteras2[1])
fifo.insert(0, lista_nodos_fronteras2[2])

In [27]:
for nodos in fifo:
    print(nodos)

<Node HanoiState: 5 4 3 2 |  | 1>
<Node HanoiState: 5 4 3 2 1 |  | >
<Node HanoiState: 5 4 3 | 1 | 2>
<Node HanoiState: 5 4 3 2 |  | 1>


- **Top(frontier)**: Podemos ver cual es nuestro siguiente nodo sin sacarlo:

In [28]:
print(f"El siguiente nodo que podemos sacar es {fifo[-1]}")

El siguiente nodo que podemos sacar es <Node HanoiState: 5 4 3 2 |  | 1>


El cual es el segundo nodo que introducimos cuando introducimos la frontera de la raíz. Como vemos, se está respetando el primero que entra, es el primero que sale.

### LIFO

Ahora veamos cómo podemos implementar un **stack**. Esto lo podemos hacer también con una lista.

In [29]:
lifo = []

- **Add(Frontier)**: Inserta el nodo en su correspondiente lugar de la cola. En el caso de la LIFO, se inserta de forma apilada para que el último que se inserte, esté listo para salir. Esto lo hacemos usando `append()`:

In [30]:
lifo.append(lista_nodos_fronteras[0])
lifo.append(lista_nodos_fronteras[1])

for nodos in lifo:
    print(nodos)

<Node HanoiState: 5 4 3 2 | 1 | >
<Node HanoiState: 5 4 3 2 |  | 1>


OBS: Los nodos están ordenados de forma inversa al caso de la fila FIFO.

- **Is-empty(frontier)**: Retorna `True` si no hay nodos en la frontera. En el caso de la implementación de una lista, podemos preguntar si la cantidad de elemento es cero.

In [31]:
print("El stack está vacio?")
if len(lifo) == 0:
    print("El stack esta vacio")
else:
    print("El stack tiene elementos")

El stack está vacio?
El stack tiene elementos


- **Pop(frontier)**: Quita el primer nodo en la cola. Con las listas tenemos el método `pop()`.

In [32]:
new_node = lifo.pop()
print(f"El nodo que sacamos es {new_node}")

El nodo que sacamos es <Node HanoiState: 5 4 3 2 |  | 1>


Podemos ver que una vez sacado el nodo, el mismo no está en la fila, y que particularmente sacamos el último nodo que entró:

In [33]:
print("Los nodos que quedan en la fila son:")
print(lifo)

Los nodos que quedan en la fila son:
[<Node HanoiState: 5 4 3 2 | 1 | >]


Si expandimos la frontera del nuevo nodo que tenemos, podemos agregarlo a la cola:

In [34]:
lista_nodos_fronteras2 = new_node.expand(problem=problem)

lifo.append(lista_nodos_fronteras2[0])
lifo.append(lista_nodos_fronteras2[1])
lifo.append(lista_nodos_fronteras2[2])

In [35]:
for nodos in lifo:
    print(nodos)

<Node HanoiState: 5 4 3 2 | 1 | >
<Node HanoiState: 5 4 3 | 2 | 1>
<Node HanoiState: 5 4 3 2 1 |  | >
<Node HanoiState: 5 4 3 2 | 1 | >


- **Top(frontier)**: Podemos ver cual es nuestro siguiente nodo sin sacarlo:

In [36]:
print(f"El siguiente nodo que podemos sacar es {lifo[-1]}")

El siguiente nodo que podemos sacar es <Node HanoiState: 5 4 3 2 | 1 | >


El cual es el último nodo que introducimos cuando introducimos la frontera del nodo que sacamos de la cola. Como vemos, se está respetando el último que entra, es el primero que sale.

### Cola prioritaria

Por último nos queda la cola prioritaria. Para ello, usaremos la librería `queue` que viene por defecto en Python. [Esta librería](https://docs.python.org/3/library/queue.html) implementa varias colas (incluida las colas FIFO o LIFO). Particularmente vamos a usar la clase `PriorityQueue`.

In [37]:
from queue import PriorityQueue

priority_queue = PriorityQueue()

- **Add(Frontier)**: Para esta cola, se debe pasar el nodo queremos encolar, pero además debemos pasar un numero para que defina la prioridad en una tupla. Esta cola funciona siempre extrayendo el nodo que tiene este valor el más chico.

In [38]:
priority_queue.put((4, lista_nodos_fronteras[0]))
priority_queue.put((2, lista_nodos_fronteras[1]))

Como vimos en clase, el valor que podemos guardar puede ser el costo hasta ir hacia ese nodo, usar una heurística, o una combinación de ambas. En este caso usamos un simple número definido arbitrariamente para ver el funcionamiento.

- **Is-empty(frontier)**: Retorna `True` si no hay nodos en la frontera. En este caso, podemos preguntarle a la cola si está vacia usando el método `empty()`:

In [39]:
print("La cola prioritaria está vacia?")
if priority_queue.empty():
    print("La cola prioritaria esta vacia")
else:
    print("La cola prioritaria tiene elementos")

La cola prioritaria está vacia?
La cola prioritaria tiene elementos


- **Pop(frontier)**: Quita el primer nodo en la cola. Con este caso tenemos el método `get()`. El nodo que va a ser extraido es el nodo con el valor de prioridad mas bajo.

In [40]:
priority_value, new_node = priority_queue.get()
print(f"El nodo que sacamos es {new_node}, cuyo valor de prioridad es {priority_value}")

El nodo que sacamos es <Node HanoiState: 5 4 3 2 |  | 1>, cuyo valor de prioridad es 2


Si expandimos la frontera del nuevo nodo que tenemos, podemos agregarlo a la cola:

In [41]:
lista_nodos_fronteras2 = new_node.expand(problem=problem)

priority_queue.put((1, lista_nodos_fronteras2[0]))
priority_queue.put((24, lista_nodos_fronteras2[1]))
priority_queue.put((54, lista_nodos_fronteras2[2]))

- **Top(frontier)**: Podemos ver cual es nuestro siguiente nodo sin sacarlo. Acá tenemos que hacer un pequeño hack dado que no es algo que tiene implementado directamente la cola:

In [42]:
print(f"El siguiente nodo que podemos sacar es {priority_queue.queue[0][-1]}, el cual tiene prioridad {priority_queue.queue[0][0]}")

El siguiente nodo que podemos sacar es <Node HanoiState: 5 4 3 | 2 | 1>, el cual tiene prioridad 1


Ahora también tenemos implementado una cola prioritaria la cual puede guardarse una función que calcula este valor de prioridad, de tal forma que en alguna implementación, ya podemos pasar directamente la función y no tengamos que calcular nosotros que valor de prioridad le corresponde a cada nodo:

In [43]:
from aima_libs.aima import PriorityQueue as AimaPriorityQueue

priority_queue2 = AimaPriorityQueue()

Ahora para usar esta cola podemos definir que la prioridad sea por el menor valor de prioridad o mayor. Y además permite obtener mediante una función que tiene como entrada al nodo, devolver el valor de la prioridad, acá podemos definir:

- Una función que devuelva el costo que lleva ir de un nodo padre a ese hijo en particular.
- Una función heurística que estime que tan lejos o fuera está de la solución.
- Una función que combine el costo que llevo ir hasta ese nodo en particular y que tan lejos de la solución se esté

Para este caso, a modo de ejemplo, vamos a implementar una función que obtenga una prioridad al azar: 

In [44]:
import random

def priority_func(x):
    return random.randint(1, 1000)

In [45]:
priority_queue2 = AimaPriorityQueue(order='min', f=priority_func)

- **Add(Frontier)**: Para esta cola, se debe pasar el nodo queremos encolar usando el método `append()`:

In [46]:
priority_queue2.append(lista_nodos_fronteras[0])
priority_queue2.append(lista_nodos_fronteras[1])

Además tenemos el método `extend()` que nos permite guardar todos los nodos de una lista directamente:  

In [47]:
priority_queue2.extend(lista_nodos_fronteras2)

Si vemos los elementos dentro de la cola, se observa:

In [48]:
priority_queue2.heap.queue

[(31, <Node HanoiState: 5 4 3 | 2 | 1>),
 (226, <Node HanoiState: 5 4 3 2 1 |  | >),
 (119, <Node HanoiState: 5 4 3 2 |  | 1>),
 (582, <Node HanoiState: 5 4 3 2 | 1 | >),
 (889, <Node HanoiState: 5 4 3 2 | 1 | >)]

- **Is-empty(frontier)**: Retorna `True` si no hay nodos en la frontera. En el caso de esta implementación, podemos preguntar si la cantidad de elementos es cero.

In [49]:
print("La cola prioritaria está vacia?")
if len(priority_queue2) == 0:
    print("La cola prioritaria esta vacia")
else:
    print("La cola prioritaria tiene elementos")

La cola prioritaria está vacia?
La cola prioritaria tiene elementos


- **Pop(frontier)**: Quita el primer nodo en la cola. Con este caso tenemos el método `pop()`. El nodo que va a ser extraido es el nodo con el valor de prioridad mas bajo.

In [50]:
priority_value, new_node = priority_queue2.pop()
print(f"El nodo que sacamos es {new_node}, cuyo valor de prioridad es {priority_value}")

El nodo que sacamos es <Node HanoiState: 5 4 3 | 2 | 1>, cuyo valor de prioridad es 31


- **Top(frontier)**: Podemos ver cual es nuestro siguiente nodo sin sacarlo. Para ello podemos usar el método `peek()`:

In [51]:
print(f"El siguiente nodo que podemos sacar es {priority_queue2.peek()[-1]}, el cual tiene prioridad {priority_queue2.peek()[0]}")

El siguiente nodo que podemos sacar es <Node HanoiState: 5 4 3 2 |  | 1>, el cual tiene prioridad 119


En todo lo que hemos visto hasta ahora, desde cómo definimos el problema hasta como se define la cola son ejemplos de implementación, los cuales son libres de usarlos o implementar sus propias soluciones

----

## Búsqueda primero en anchura

Con todo lo que fuimos implementando, ahora podemos aplicar un algoritmo de búsqueda. Obsérvese todo lo que tuvimos que previamente definir e implementar para poder llegar hasta aquí. 

Vamos a implementar el algoritmo de búsqueda **búsqueda primero en anchura**, tal como vimos en el video, arranca desde la raíz y va expandiendo todos los nodos nivel a nivel usando una cola FIFO.

![breadth_first_search](./img/breadth_first_search.png)

Empecemos con menos discos para probar cómo implementarlo y luego vayamos al caso con 5 discos:

In [52]:
# Inicializamos el problema
initial_state = StatesHanoi([3, 2, 1], [], [], max_disks=3)
goal_state = StatesHanoi([], [], [3, 2, 1], max_disks=3)
problem = ProblemHanoi(initial=initial_state, goal=goal_state)

frontier = [NodeHanoi(problem.initial)]  # Creamos una cola FIFO con el nodo inicial

find_solution = False

# Mientras que la cola no este vacia
while len(frontier) != 0:
    node = frontier.pop()  # Extraemos el primer nodo de la cola
    if problem.goal_test(node.state):  # Comprobamos si hemos alcanzado el estado objetivo
        last_node = node
        find_solution = True
        print("Encontramos la solución")
        break
    
    # Agregamos a la cola todos los nodos sucesores del nodo actual
    for next_node in node.expand(problem):
        frontier.insert(0, next_node)

if not find_solution:
    print("No se encontró la solución")

Encontramos la solución


Una vez que encontramos la solución podemos ver, analizando el ultimo nodo del arbol, cuanto nos costo llegar a la solución:

In [53]:
print(f'Longitud del camino de la solución: {last_node.state.accumulated_cost}')
print(f'Profundidad del arbol donde se encontró la solución: {last_node.depth}')

Longitud del camino de la solución: 7.0
Profundidad del arbol donde se encontró la solución: 7


Veamos ahora con 4 discos:

In [54]:
# Inicializamos el problema
initial_state = StatesHanoi([4, 3, 2, 1], [], [], max_disks=4)
goal_state = StatesHanoi([], [], [4, 3, 2, 1], max_disks=4)
problem = ProblemHanoi(initial=initial_state, goal=goal_state)

find_solution = False

# Mientras que la cola no este vacia
while len(frontier) != 0:
    node = frontier.pop()  # Extraemos el primer nodo de la cola
    if problem.goal_test(node.state):  # Comprobamos si hemos alcanzado el estado objetivo
        last_node = node
        find_solution = True
        print("Encontramos la solución")
        break
    
    # Agregamos a la cola todos los nodos sucesores del nodo actual
    for next_node in node.expand(problem):
        frontier.insert(0, next_node)

if not find_solution:
    print("No se encontró la solución")

KeyboardInterrupt: 

Este algoritmo lleva mucho tiempo (pueden interrumpir la ejecución), porque está ocurriendo un fenómeno que vimos previamente cuando analizamos el árbol: 

![tree](./img/tree_hanoi3.png)

Cuando expandimos un nodo de la frontera, los nuevos nodos pueden ser de estados que ya fueron explorados en previos nodos, lo que nos hace demorar y entrar en bucles prácticamente infinitos 

Entonces armemos una variante que consuma más memoria pero que guarde los estados que ya fueron explorados, por lo que si un nodo es de un estado que ya fue explorado, no lo ponemos en la cola. Para poder hacer eso, usamos **set** de Python, el cual es una estructura de datos ideal para estos casos:  

In [55]:
# Inicializamos el problema
initial_state = StatesHanoi([4, 3, 2, 1], [], [], max_disks=4)
goal_state = StatesHanoi([], [], [4, 3, 2, 1], max_disks=4)
problem = ProblemHanoi(initial=initial_state, goal=goal_state)

frontier = [NodeHanoi(problem.initial)]  # Creamos una cola FIFO con el nodo inicial

explored = set()  # Este set nos permite ver si ya exploramos un estado para evitar repetir indefinidamente

find_solution = False
node_explored = 0

# Mientras que la cola no este vacia
while len(frontier) != 0:
    node = frontier.pop()  # Extraemos el primer nodo de la cola
    node_explored += 1
    
    # Agregamos el estado del nodo al set. Esto evita guardar duplicados, porque set nunca tiene elementos repetidos
    explored.add(node.state)
    
    if problem.goal_test(node.state):  # Comprobamos si hemos alcanzado el estado objetivo
        last_node = node
        find_solution = True
        print("Encontramos la solución")
        break
    
    # Agregamos a la cola todos los nodos sucesores del nodo actual
    for next_node in node.expand(problem):
        # Solo si el estado del nodo no fue explorado
        if next_node.state not in explored:
            frontier.insert(0, next_node)

if not find_solution:
    print("No se encontró la solución")

Encontramos la solución


Una vez que encontramos la solución podemos ver, analizando el último nodo del árbol, cuanto nos costo llegar a la solución:

In [56]:
print(f'Longitud del camino de la solución: {last_node.state.accumulated_cost}')
print(f'Profundidad del arbol donde se encontró la solución: {last_node.depth}')

Longitud del camino de la solución: 15.0
Profundidad del arbol donde se encontró la solución: 15


Como todo paso tiene el mismo costo, la **búsqueda primero en anchura** nos asegura que el camino es más corto, que este caso, al ser el costo unitario, corresponde a la minima profundidad del árbol que se encontró la solución.

Además, podemos ver cuántos nodos se exploraron, cuánto estados diferentes corresponden y cuantos nodos nos quedaron en la cola,

In [57]:
print(node_explored, "nodos se expandieron, correspondientes a", len(explored),"estados diferentes y", len(frontier), "nodos quedaron en la frontera")

216 nodos se expandieron, correspondientes a 71 estados diferentes y 52 nodos quedaron en la frontera


Podemos ver el camino tomado que nos llevo a la solución:

In [58]:
for nodos in last_node.path():
    print(nodos)

<Node HanoiState: 4 3 2 1 |  | >
<Node HanoiState: 4 3 2 | 1 | >
<Node HanoiState: 4 3 | 1 | 2>
<Node HanoiState: 4 3 |  | 2 1>
<Node HanoiState: 4 | 3 | 2 1>
<Node HanoiState: 4 1 | 3 | 2>
<Node HanoiState: 4 1 | 3 2 | >
<Node HanoiState: 4 | 3 2 1 | >
<Node HanoiState:  | 3 2 1 | 4>
<Node HanoiState:  | 3 2 | 4 1>
<Node HanoiState: 2 | 3 | 4 1>
<Node HanoiState: 2 1 | 3 | 4>
<Node HanoiState: 2 1 |  | 4 3>
<Node HanoiState: 2 | 1 | 4 3>
<Node HanoiState:  | 1 | 4 3 2>
<Node HanoiState:  |  | 4 3 2 1>


Y las acciones que el agente deberia aplicar para llegar a la solución es:

In [59]:
for act in last_node.solution():
    print(act)

Move disk 1 from 1 to 2
Move disk 2 from 1 to 3
Move disk 1 from 2 to 3
Move disk 3 from 1 to 2
Move disk 1 from 3 to 1
Move disk 2 from 3 to 2
Move disk 1 from 1 to 2
Move disk 4 from 1 to 3
Move disk 1 from 2 to 3
Move disk 2 from 2 to 1
Move disk 1 from 3 to 1
Move disk 3 from 2 to 3
Move disk 1 from 1 to 2
Move disk 2 from 1 to 3
Move disk 1 from 2 to 3


Ya que tenemos algo que funciona, creemos una función que aplique el algoritmo **búsqueda primero en anchura**, el cuál nos retorna la solución y algunas metricas de la ejecución:

In [60]:
def breadth_first_search(number_disks=5):
    # Inicializamos el problema
    list_disks = [i for i in range(5, 0, -1)]
    initial_state = StatesHanoi(list_disks, [], [], max_disks=number_disks)
    goal_state = StatesHanoi([], [], list_disks, max_disks=number_disks)
    problem = ProblemHanoi(initial=initial_state, goal=goal_state)

    # Creamos una cola FIFO con el nodo inicial
    frontier = [NodeHanoi(problem.initial)]  

    # Creamos el set con estados ya visitados
    explored = set()
    
    node_explored = 0
    
    while len(frontier) != 0:
        node = frontier.pop()
        node_explored += 1
        
        # Agregamos el estado del nodo al set. Esto evita guardar duplicados, porque set nunca tiene elementos repetidos
        explored.add(node.state)
        
        if problem.goal_test(node.state):  # Comprobamos si hemos alcanzado el estado objetivo
            metrics = {
                "solution_found": True,
                "nodes_explored": node_explored,
                "states_visited": len(explored),
                "nodes_in_frontier": len(frontier),
                "max_depth": node.depth,
                "cost_total": node.state.accumulated_cost,
            }
            return node, metrics
        
        # Agregamos a la cola todos los nodos sucesores del nodo actual
        for next_node in node.expand(problem):
            # Solo si el estado del nodo no fue explorado
            if next_node.state not in explored:
                frontier.insert(0, next_node)

    # Si no se encontro la solución, devolvemos la métricas igual
    metrics = {
        "solution_found": False,
        "nodes_explored": node_explored,
        "states_visited": len(explored),
        "nodes_in_frontier": len(frontier),
        "max_depth": node.depth, # OBS: Si no se encontró la solución, este valor solo tiene sentido en breadth_first_search, en otros casos se debe ir llevando registro de cual fue la máxima profundidad
        "cost_total": None,
    }
    return None, metrics

Implementemos con los 5 discos:

In [61]:
solution, metrics = breadth_first_search(number_disks=5)

Veamos algunas metricas:

In [62]:
for key, value in metrics.items():
    print(f"{key}: {value}")

solution_found: True
nodes_explored: 1351
states_visited: 233
nodes_in_frontier: 285
max_depth: 31
cost_total: 31.0


Veamos ahora a la solución:

In [63]:
for nodos in solution.path():
    print(nodos)

<Node HanoiState: 5 4 3 2 1 |  | >
<Node HanoiState: 5 4 3 2 |  | 1>
<Node HanoiState: 5 4 3 | 2 | 1>
<Node HanoiState: 5 4 3 | 2 1 | >
<Node HanoiState: 5 4 | 2 1 | 3>
<Node HanoiState: 5 4 1 | 2 | 3>
<Node HanoiState: 5 4 1 |  | 3 2>
<Node HanoiState: 5 4 |  | 3 2 1>
<Node HanoiState: 5 | 4 | 3 2 1>
<Node HanoiState: 5 | 4 1 | 3 2>
<Node HanoiState: 5 2 | 4 1 | 3>
<Node HanoiState: 5 2 1 | 4 | 3>
<Node HanoiState: 5 2 1 | 4 3 | >
<Node HanoiState: 5 2 | 4 3 | 1>
<Node HanoiState: 5 | 4 3 2 | 1>
<Node HanoiState: 5 | 4 3 2 1 | >
<Node HanoiState:  | 4 3 2 1 | 5>
<Node HanoiState: 1 | 4 3 2 | 5>
<Node HanoiState: 1 | 4 3 | 5 2>
<Node HanoiState:  | 4 3 | 5 2 1>
<Node HanoiState: 3 | 4 | 5 2 1>
<Node HanoiState: 3 | 4 1 | 5 2>
<Node HanoiState: 3 2 | 4 1 | 5>
<Node HanoiState: 3 2 1 | 4 | 5>
<Node HanoiState: 3 2 1 |  | 5 4>
<Node HanoiState: 3 2 |  | 5 4 1>
<Node HanoiState: 3 | 2 | 5 4 1>
<Node HanoiState: 3 | 2 1 | 5 4>
<Node HanoiState:  | 2 1 | 5 4 3>
<Node HanoiState: 1 | 2 | 5 4 

Y las acciones que el agente deberia aplicar para llegar a la solución es:

In [64]:
for act in solution.solution():
    print(act)

Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from 1 to 3
Move disk 1 from 2 to 1
Move disk 2 from 2 to 3
Move disk 1 from 1 to 3
Move disk 4 from 1 to 2
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 1
Move disk 3 from 3 to 2
Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 5 from 1 to 3
Move disk 1 from 2 to 1
Move disk 2 from 2 to 3
Move disk 1 from 1 to 3
Move disk 3 from 2 to 1
Move disk 1 from 3 to 2
Move disk 2 from 3 to 1
Move disk 1 from 2 to 1
Move disk 4 from 2 to 3
Move disk 1 from 1 to 3
Move disk 2 from 1 to 2
Move disk 1 from 3 to 2
Move disk 3 from 1 to 3
Move disk 1 from 2 to 1
Move disk 2 from 2 to 3
Move disk 1 from 1 to 3


## Medición de performance

Por último, ante la pregunta de cómo podemos medir la performance del algoritmo de búsqueda, tenemos un par de herramientas de profiling que nos da Python.

Primero, si queremos medir tiempo, podemos hacer uso de `%%timeit` que nos permite medir tiempo de las celdas de Jupyter:

In [65]:
%%timeit 
solution, metrics = breadth_first_search(number_disks=5)

61.2 ms ± 1.59 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Vemos que la solución demoró en promedio 60 ms. Esto va variar de máquina a máquina y sólo tendrá sentido comparar entre diferente algoritmos de búsqueda si se usa la misma máquina.
 
Medir memoria es un poco más difícil, algo que podemos hacer es medir el pico de consumo de memoria RAM, el cual nos va dar una idea de cuanta memoria RAM se consumió mientras ocurre el proceso. Para ello usamos `tracemalloc`:

In [66]:
import tracemalloc

tracemalloc.start()

solution, metrics = breadth_first_search(number_disks=5)

# Para medir memoria consumida usamos el pico de memoria
_, memory_peak = tracemalloc.get_traced_memory()
memory_peak /= 1024*1024
tracemalloc.stop()

print(f"Pico de memoria ocupada: {round(memory_peak, 2)} [MB]", )

Pico de memoria ocupada: 1.61 [MB]
